In [1]:
source("~/software/notebook_assist/functions.R")
source("~/software/notebook_assist/packages.R")

setwd("../")
list.files()


[1] "figures"             "input"               "notebooks"          
[4] "source_data"         "supplemental_tables"

In [23]:

### Load HOCOMOCO and JASPAR Metadata
hocomoco   = fread("input/figure1/HOCOMOCOv11_core_annotation_HUMAN_mono.tsv",sep="\t",data.table=F)[,1:2]

# jaspar     = 

### TFBS Predictions can be found on Figshare
#### JASPAR
ipsc.jaspar = fread("input/figure1/iPSC_JAPSAR_TOBIAS.txt",sep="\t",data.table=F)
cvpc.jaspar = fread("input/figure1/CVPC_JAPSAR_TOBIAS.txt",sep="\t",data.table=F)
ppc.jaspar  = fread("input/figure1/PPC_JAPSAR_TOBIAS.txt",sep="\t",data.table=F)
all.jaspar    = as.data.frame(rbindlist(list(ipsc.jaspar,cvpc.jaspar,ppc.jaspar)))
all.jaspar$Tissue = ifelse(grepl("ipsc",all.jaspar$PeakID), "iPSC",
                         ifelse(grepl("cvpc",all.jaspar$PeakID), "CVPC","PPC"))

#### HOCOMOCO
ipsc.hoco   = fread("input/figure1/iPSC_HOCOMOCO_TOBIAS.txt",sep="\t",data.table=F)
cvpc.hoco   = fread("input/figure1/CVPC_HOCOMOCO_TOBIAS.txt",sep="\t",data.table=F)
ppc.hoco    = fread("input/figure1/PPC_HOCOMOCO_TOBIAS.txt",sep="\t",data.table=F)
all.hoco    = as.data.frame(rbindlist(list(ipsc.hoco,cvpc.hoco,ppc.hoco)))
all.hoco$Tissue = ifelse(grepl("ipsc",all.hoco$PeakID), "iPSC",
                         ifelse(grepl("cvpc",all.hoco$PeakID), "CVPC","PPC"))
###Load Three Sets of ATAC Peaks
ipsc.atac.path = "input/figure1/ipsc_atac_peaks.bed"
cvpc.atac.path = "input/figure1/cvpc_atac_peaks.bed"
ppc.atac.path  = "input/figure1/ppc_atac_peaks.bed"


In [47]:
length(unique(ipsc.jaspar$MotifID))

[1] 746

In [27]:
tissues = c("ipsc","cvpc","ppc")

In [30]:


message(command1)

bedtools intersect -a input/figure1/ipsc_atac_peaks.bed -b input/figure1/cvpc_atac_peaks.bed -wb > input/figure1/ipsc_cvpc.bed



In [37]:

command1 = paste("bedtools", "intersect", "-a", ipsc.atac.path,"-b", cvpc.atac.path, "-wb", "> input/figure1/ipsc_cvpc.bed")
command2 = paste("bedtools", "intersect", "-a", ipsc.atac.path,"-b", ppc.atac.path, "-wb", "> input/figure1/ipsc_ppc.bed")
command3 = paste("bedtools", "intersect", "-a", cvpc.atac.path,"-b", ppc.atac.path, "-wb", "> input/figure1/cvpc_ppc.bed")

### fix before upload

shared_peaks_bed   = as.data.frame(rbindlist(list(fread("input/figure1/ipsc_cvpc.bed",sep="\t",data.table=F)[,c(4,8)],
                                                  fread("input/figure1/ipsc_ppc.bed",sep="\t",data.table=F)[,c(4,8)],
                                                  fread("input/figure1/cvpc_ppc.bed",sep="\t",data.table=F)[,c(4,8)])))
shared_peaks     = unique(c(shared_peaks_bed[[1]],shared_peaks_bed[[2]]))
length(shared_peaks)
# # fread(cmd = command, sep="\t",data.table=F)

[1] 356862

In [38]:
all.peaks = as.data.frame(rbindlist(lapply(c(ipsc.atac.path,cvpc.atac.path,ppc.atac.path),
                                          function(x)return(fread(x,sep="\t")))))

all.peaks$Tissue = ifelse(grepl("ipsc",all.peaks$V4), "iPSC",
                         ifelse(grepl("cvpc",all.peaks$V4), "CVPC","PPC"))
names(all.peaks)[4] = "PeakID"
                                              
cast_tf_annotations = function(tfbs_df){
    all.peaks$specific   = ifelse(all.peaks$PeakID %in% shared_peaks, 0, 1)
    peak_tfs         = unique(tfbs_df[,1:2])
    peak_tfs$TF      = gsub("_HUMAN.*","",gsub("_MA.*","",peak_tfs[,1]))
    peak_tfs$Bound   = 1
    tf_cast          = merge(all.peaks[,c("PeakID","specific")],
                             reshape2::dcast(PeakID ~ TF, data = peak_tfs, value.var = "Bound", fun.aggregate = sum),
                             all=TRUE)
    tf_cast[ is.na(tf_cast)] = 0 
    return(tf_cast)
    }
                                              
all.hoco.cast = cast_tf_annotations(all.hoco)
all.jaspar.cast = cast_tf_annotations(all.jaspar)
                                              



In [59]:
tmp = all.jaspar.cast[ grepl("ipsc",all.jaspar.cast$PeakID) & rowSums(data.matrix(all.jaspar.cast[,3:ncol(all.jaspar.cast)])) > 0,]
table(tmp$specific)


    0     1 
40632  5110 

In [42]:
calculate_tissue_tfs = function(tissue, tf_cast_df) {
    tf_cast        = tf_cast_df[ grepl(tissue,tf_cast_df$PeakID) & rowSums(data.matrix(tf_cast_df[,3:ncol(tf_cast_df)])) > 0,]

    property = colnames(tf_cast[,3:ncol(tf_cast)])
    calculate_enrichment = function(property, vertices){
        totest          = vertices[, c(property, "specific")]

        colnames(totest) = c("property", "annot")

        annots = names(table(totest$annot))
    
        tests = as.data.frame(rbindlist(lapply(annots, function(annot){
            x11  = nrow(totest[ totest$property == 1 & totest$annot == annot, ])
            x12  = nrow(totest[ totest$property == 1 & totest$annot != annot, ])
            x21  = nrow(totest[ totest$property == 0 & totest$annot == annot, ])
            x22  = nrow(totest[ totest$property == 0 & totest$annot != annot, ])
            mat  = matrix(c(x11, x12, x21, x22), nrow = 2, byrow = TRUE)
            test = fisher.test(mat)
            out  = data.frame(TF = property, Specific = annot, 
                              x11 = x11, x12 = x12, x21 = x21, x22 = x22, estimate = test$estimate, 
                              ci1 = test$conf.int[[1]], ci2 = test$conf.int[[2]], pval = test$p.value)
            return(out)
        })), stringsAsFactors = FALSE)
        return(tests)}
    enrich = function(x, annot){
        tests       = as.data.frame(rbindlist(lapply(1:length(property), 
                                                     function(ii){calculate_enrichment(property[ii], x)})))
        tests       = tests[ tests$Specific == 1,]
        tests$padj  = p.adjust(tests$pval, method = "BH")
        tests$log2r = log2(tests$estimate)
        return(tests)}
    enrich_out = enrich(tf_cast)
    enrich_out$Specific = tissue
    return(enrich_out)
}

jaspar_specific_enrich = as.data.frame(rbindlist(lapply(c("ipsc","cvpc","ppc"),
                                                        calculate_tissue_tfs,
                                                        tf_cast_df = all.jaspar.cast)))

hoco_specific_enrich = as.data.frame(rbindlist(lapply(c("ipsc","cvpc","ppc"),
                                                        calculate_tissue_tfs,
                                                        tf_cast_df = all.hoco.cast)))

fwrite(jaspar_specific_enrich, "source_data/jaspar_specific_enrich.txt",
       sep="\t",row.names=F,quote=F)
fwrite(hoco_specific_enrich, "source_data/hoco_specific_enrich.txt",
       sep="\t",row.names=F,quote=F)

In [43]:
jaspar_specific_enrich = fread("source_data/jaspar_specific_enrich.txt",sep="\t",data.table=F)
hoco_specific_enrich   = fread("source_data/hoco_specific_enrich.txt",sep="\t",data.table=F)


shared_enrich = function(row, enrich_df){
    x11 = enrich_df[row,]$x12
    x12 = enrich_df[row,]$x22
    x21 = enrich_df[row,]$x11
    x22 = enrich_df[row,]$x21
    
    mat = matrix(c(x11,x12,x21,x22), nrow=2)
    test = fisher.test(mat)
    out  = data.frame(TF = enrich_df[row,]$TF, Specific = paste0(enrich_df[row,]$Specific,"_shared"),
                              x11 = x11, x12 = x12, x21 = x21, x22 = x22, estimate = test$estimate, 
                              ci1 = test$conf.int[[1]], ci2 = test$conf.int[[2]], pval = test$p.value)
    return(out)   
}
jaspar_shared_enrich = as.data.frame(rbindlist(lapply(1:nrow(jaspar_specific_enrich),
                                                      shared_enrich,
                                                      enrich_df = jaspar_specific_enrich))) %>% 
                       group_by(Specific) %>% mutate(padj = p.adjust(pval,method="BH"), log2r = log2(estimate)) 

hoco_shared_enrich = as.data.frame(rbindlist(lapply(1:nrow(hoco_specific_enrich),
                                                    shared_enrich, 
                                                    enrich_df = hoco_specific_enrich))) %>% 
                       group_by(Specific) %>% mutate(padj = p.adjust(pval,method="BH"), log2r = log2(estimate)) 

In [44]:
to_supplement = function(enrich_df,motif_db) {
    enrich_df$Tissue    = ifelse(grepl("ipsc",enrich_df$Specific), "iPSC",
                             ifelse(grepl("cvpc",enrich_df$Specific), "CVPC","PPC"))
    enrich_df$Type      = ifelse(grepl("shared",enrich_df$Specific), "Shared", "Specific")
    enrich_df$Database  = motif_db
    supp_out = enrich_df[,c("TF","Database","Tissue","Type","estimate",
                           "ci1","ci2","pval","padj","log2r")]
    colnames(supp_out) = c("TF","Database","Tissue","Type","Odds Ratio",
                           "Low Confidence Interval","High Confidence Interval",
                           "P-value","Adjusted P-value","log2(OR)")
    return(supp_out)
}

fwrite(rbind(to_supplement(rbind(jaspar_specific_enrich,jaspar_shared_enrich),"JASPAR"),
        to_supplement(rbind(hoco_specific_enrich,hoco_shared_enrich),"HOCOMOCO")),
       "supplemental_tables/Table_S5.txt",
       sep="\t",row.names=F,quote=F)


In [84]:
table5 = fread("supplemental_tables/Table_S5.txt",sep="\t",data.table=F)
table5_hoco = table5[ table5$Database == "HOCOMOCO",]
length(unique(table5_hoco$TF))
hocomoco2 = hocomoco

colnames(hocomoco2) = c("MotifID","AltID")
hocomoco2$TF = gsub("_HUMAN.*","",hocomoco2$MotifID)
hocomoco2$Database = "HOCOMOCO"

table5_jaspar = table5[ table5$Database == "JASPAR",]


jaspar.df = data.frame(MotifID = unique(ipsc.jaspar$MotifID))
jaspar.df$TF = gsub("_MA.*","",jaspar.df$MotifID)
jaspar.df$AltID = NA
jaspar.df$Database = "JASPAR"
key = rbind(jaspar.df,hocomoco2)
table5a = merge(table5, key,by=c("TF","Database"))
colnames(table5a)
fwrite(table5a[,c(11,1,12,2:10)],"~/projects/Resource/manuscript/supplemental_tables/Table_S5.txt",
       sep="\t",row.names=F,quote=F)

[1] 401

[1] "TF"                       "Database"                
 [3] "Tissue"                   "Type"                    
 [5] "Odds Ratio"               "Low Confidence Interval" 
 [7] "High Confidence Interval" "P-value"                 
 [9] "Adjusted P-value"         "log2(OR)"                
[11] "MotifID"                  "AltID"

In [29]:
table5a = merge(table5, rbind(hocomoco2,jaspar.df),by=c("TF","Database"))
nrow(table5a)
nrow(table5)

[1] 6090

[1] 6882

In [38]:
setdiff(table5$TF, table5$TF)

character(0)

In [37]:
hocomoco

Model,Transcription factor
<chr>,<chr>
AHR_HUMAN.H11MO.0.B,AHR
AIRE_HUMAN.H11MO.0.C,AIRE
ALX1_HUMAN.H11MO.0.B,ALX1
ANDR_HUMAN.H11MO.0.A,AR
AP2A_HUMAN.H11MO.0.A,TFAP2A
AP2B_HUMAN.H11MO.0.B,TFAP2B
AP2C_HUMAN.H11MO.0.A,TFAP2C
ARI5B_HUMAN.H11MO.0.C,ARID5B
ARNT_HUMAN.H11MO.0.B,ARNT
